# Assignment 5, Question 6: Data Transformation

**Points: 20**

Transform and engineer features from the clinical trial dataset.

## Setup

In [221]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import utilities
from q3_data_utils import load_data, clean_data, transform_types, create_bins, fill_missing

df = load_data('output/q5_cleaned_data.csv')
print(f"Loaded {len(df)} patients")

# Prewritten visualization functions for transformation analysis
def plot_distribution(series, title, figsize=(10, 6)):
    """
    Create a histogram of a numeric series.
    
    Args:
        series: pandas Series with numeric data
        title: Chart title
        figsize: Figure size tuple
    """
    plt.figure(figsize=figsize)
    series.hist(bins=30)
    plt.title(title)
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.tight_layout()
    plt.show()

def plot_value_counts(series, title, figsize=(10, 6)):
    """
    Create a bar chart of value counts.
    
    Args:
        series: pandas Series with value counts
        title: Chart title
        figsize: Figure size tuple
    """
    plt.figure(figsize=figsize)
    series.plot(kind='bar')
    plt.title(title)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

Loaded 10000 patients


## Part 1: Type Conversions (5 points)

1. Convert 'enrollment_date' to datetime using the `transform_types()` utility
2. Convert categorical columns ('site', 'intervention_group', 'sex') to category dtype
3. Ensure all numeric columns are proper numeric types
4. Display the updated dtypes

In [222]:
# TODO: Type conversions
# 1. Use transform_types() to convert enrollment_date to datetime
# 2. Convert categorical columns ('site', 'intervention_group', 'sex') to category dtype
# 3. Ensure all numeric columns are proper numeric types
# 4. Display the updated dtypes using df.dtypes
df.columns
type_map = {
    "patient_id": 'string',
    'enrollment_date': 'datetime',
    'site': 'category',
    'intervention_group': 'category',
    'sex': 'category',
    'age': 'numeric',
    'bmi': 'numeric',
    'systolic_bp': 'numeric',
    'diastolic_bp': 'numeric', 
    'cholesterol_total': 'numeric', 
    'cholesterol_hdl': 'numeric',
    'cholesterol_ldl': 'numeric', 
    'glucose_fasting': 'numeric',
    'follow_up_months': 'numeric',
    'adherence_pct': 'numeric',
    'outcome_cvd': 'category',
    'dropout': 'category'
   }

# ??? may be take care of outcome_cvd and dropout later
df_transformed = transform_types(df, type_map)
df_transformed.dtypes

patient_id            string[python]
age                            int64
sex                         category
bmi                          float64
enrollment_date       datetime64[ns]
systolic_bp                  float64
diastolic_bp                 float64
cholesterol_total            float64
cholesterol_hdl              float64
cholesterol_ldl              float64
glucose_fasting              float64
site                        category
intervention_group          category
follow_up_months               int64
adverse_events                 int64
outcome_cvd                 category
adherence_pct                float64
dropout                     category
dtype: object

## Part 2: Feature Engineering (8 points)

Create these new calculated columns:

1. `cholesterol_ratio` = cholesterol_ldl / cholesterol_hdl
2. `bp_category` = categorize systolic BP:
   - 'Normal': < 120
   - 'Elevated': 120-129
   - 'High': >= 130
3. `age_group` using `create_bins()` utility:
   - Bins: [0, 40, 55, 70, 100]
   - Labels: ['<40', '40-54', '55-69', '70+']
4. `bmi_category` using standard BMI categories:
   - Underweight: <18.5
   - Normal: 18.5-24.9
   - Overweight: 25-29.9
   - Obese: >=30

In [223]:
# TODO: Calculate cholesterol ratio
df_transformed["cholesterol_ratio"] = df_transformed["cholesterol_total"] / df_transformed["cholesterol_hdl"]

In [224]:
# TODO: Categorize blood pressure
BP_binned_df = create_bins(df_transformed, 'systolic_bp',bins = [0, 120,130, 200], labels = ["Normal", "Elevated","High"], new_column = "bp_category")
BP_binned_df.head()

,patient_id,age,sex,bmi,enrollment_date,systolic_bp,diastolic_bp,cholesterol_total,cholesterol_hdl,cholesterol_ldl,glucose_fasting,site,intervention_group,follow_up_months,adverse_events,outcome_cvd,adherence_pct,dropout,cholesterol_ratio,bp_category
0,P00001,80,F,29.3,2022-05-01,123.0,80.0,120.0,55.0,41.0,118.0,site b,Control,20,0,No,24.0,No,2.181818,Elevated
1,P00002,80,Female,26.0,2022-01-06,139.0,81.0,206.0,58.0,107.0,79.0,Site A,CONTROL,24,0,No,77.0,No,3.551724,High
2,P00003,82,Female,-1.0,2023-11-04,123.0,86.0,172.0,56.0,82.0,77.0,SITE C,treatment b,2,0,Yes,70.0,No,3.071429,Elevated
3,P00004,95,Female,25.4,2022-08-15,116.0,77.0,200.0,56.0,104.0,115.0,Site D,treatment b,17,0,No,62.0,No,3.571429,Normal
4,P00005,95,M,26.0,2023-04-17,97.0,71.0,185.0,78.0,75.0,113.0,site e,Treatmen A,9,0,yes,62.0,Yes,2.371795,Normal


**Note:** The `create_bins()` function has an optional `new_column` parameter. If you don't specify it, the new column will be named `{original_column}_binned`. You can use `new_column='age_group'` to give it a custom name.


In [225]:
# TODO: Create age groups
age_binned_df = create_bins(BP_binned_df, 'age',bins = [0, 40, 55, 70, 100], labels = ['<40', '40-54', '55-69', '70+'], new_column = "age_group")
age_binned_df.head()

,patient_id,age,sex,bmi,enrollment_date,systolic_bp,diastolic_bp,cholesterol_total,cholesterol_hdl,cholesterol_ldl,...,site,intervention_group,follow_up_months,adverse_events,outcome_cvd,adherence_pct,dropout,cholesterol_ratio,bp_category,age_group
0,P00001,80,F,29.3,2022-05-01,123.0,80.0,120.0,55.0,41.0,...,site b,Control,20,0,No,24.0,No,2.181818,Elevated,70+
1,P00002,80,Female,26.0,2022-01-06,139.0,81.0,206.0,58.0,107.0,...,Site A,CONTROL,24,0,No,77.0,No,3.551724,High,70+
2,P00003,82,Female,-1.0,2023-11-04,123.0,86.0,172.0,56.0,82.0,...,SITE C,treatment b,2,0,Yes,70.0,No,3.071429,Elevated,70+
3,P00004,95,Female,25.4,2022-08-15,116.0,77.0,200.0,56.0,104.0,...,Site D,treatment b,17,0,No,62.0,No,3.571429,Normal,70+
4,P00005,95,M,26.0,2023-04-17,97.0,71.0,185.0,78.0,75.0,...,site e,Treatmen A,9,0,yes,62.0,Yes,2.371795,Normal,70+


In [226]:
# TODO: Create BMI categories
all_binned_df = create_bins(age_binned_df, 'bmi',bins = [0, 18.5,25, 30, 250], labels = ["Underweight", "Normal","Overweight","Obese"], new_column = "bmi_category")
all_binned_df.head()

,patient_id,age,sex,bmi,enrollment_date,systolic_bp,diastolic_bp,cholesterol_total,cholesterol_hdl,cholesterol_ldl,...,intervention_group,follow_up_months,adverse_events,outcome_cvd,adherence_pct,dropout,cholesterol_ratio,bp_category,age_group,bmi_category
0,P00001,80,F,29.3,2022-05-01,123.0,80.0,120.0,55.0,41.0,...,Control,20,0,No,24.0,No,2.181818,Elevated,70+,Overweight
1,P00002,80,Female,26.0,2022-01-06,139.0,81.0,206.0,58.0,107.0,...,CONTROL,24,0,No,77.0,No,3.551724,High,70+,Overweight
2,P00003,82,Female,-1.0,2023-11-04,123.0,86.0,172.0,56.0,82.0,...,treatment b,2,0,Yes,70.0,No,3.071429,Elevated,70+,NaN
3,P00004,95,Female,25.4,2022-08-15,116.0,77.0,200.0,56.0,104.0,...,treatment b,17,0,No,62.0,No,3.571429,Normal,70+,Overweight
4,P00005,95,M,26.0,2023-04-17,97.0,71.0,185.0,78.0,75.0,...,Treatmen A,9,0,yes,62.0,Yes,2.371795,Normal,70+,Overweight


## Part 3: String Cleaning (2 points)

If there are any string columns that need cleaning:
1. Convert to lowercase
2. Strip whitespace
3. Replace any placeholder values

In [227]:
# TODO: String cleaning
# sex,site, intervention_group, outcome_cvd, dropout
str_clean_df = all_binned_df.copy()

string_columns = ['sex','site', 'intervention_group', 'outcome_cvd', 'dropout']
str_clean_df[string_columns] = str_clean_df[string_columns].apply(lambda x: x.str.strip().str.lower())

print(str_clean_df["sex"].unique())

print(str_clean_df["site"].unique())

str_clean_df = str_clean_df.replace({"f": "female", "m": "female", 
                                     "treatmen a": "treatment a", "treatmenta": "treatment a",
                                    "treatment  b": "treatment b",
                                    "contrl": "control",
                                    "site  a": "site a",
                                    "site_d": "site d"
                                    })
str_clean_df.head()
print(str_clean_df["site"].unique())



['f' 'female' 'm' 'male']
['site b' 'site a' 'site c' 'site d' 'site e' 'site_d' 'site  a']
['site b' 'site a' 'site c' 'site d' 'site e']


## Part 4: One-Hot Encoding (5 points)

Create dummy variables for categorical columns:
1. One-hot encode 'intervention_group' using `pd.get_dummies()`
2. One-hot encode 'site'
3. Drop the original categorical columns
4. Show the new shape and column names

In [228]:
# TODO: One-hot encoding
intervention_dummies = pd.get_dummies(str_clean_df["intervention_group"],prefix='intervention', dtype='int64')
site_dummies = pd.get_dummies(str_clean_df["site"], dtype='int64')
site_dummies 

# remove original columns
str_clean_df.drop(["intervention_group", "site"],axis=1, inplace=True)

# add dummies to dataframe
df_with_dummies = pd.concat([str_clean_df, intervention_dummies,site_dummies], axis=1)

print(df_with_dummies.shape)
print(df_with_dummies.columns)

(10000, 28)
Index(['patient_id', 'age', 'sex', 'bmi', 'enrollment_date', 'systolic_bp',
       'diastolic_bp', 'cholesterol_total', 'cholesterol_hdl',
       'cholesterol_ldl', 'glucose_fasting', 'follow_up_months',
       'adverse_events', 'outcome_cvd', 'adherence_pct', 'dropout',
       'cholesterol_ratio', 'bp_category', 'age_group', 'bmi_category',
       'intervention_control', 'intervention_treatment a',
       'intervention_treatment b', 'site a', 'site b', 'site c', 'site d',
       'site e'],
      dtype='object')


## Part 5: Save Transformed Data

Save the fully transformed dataset to `output/q6_transformed_data.csv`

In [229]:
# TODO: Save transformed data
# df_transformed.to_csv('output/q6_transformed_data.csv', index=False)
df_with_dummies.to_csv('output/q6_transformed_data.csv', index=False)